# Layer.ai Air Quality Prediction Challenge
# By Mohamed Eltayeb & Azer Ksouri

# Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_absolute_error

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', None)

## Define Functions

In [ ]:
#Plot the LGBM Features Importances
def plotImp(model, X , num = 20, fig_size = (40, 20)):
    feature_imp = pd.DataFrame({'Value':model.feature_importances_,'Feature':X.columns})
    plt.figure(figsize=fig_size)
    sns.set(font_scale = 5)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", 
                                                        ascending=False)[0:num])
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances-01.png')
    plt.show()

In [ ]:
#Label Encoder
def label_enc(train_df, test_df, features):
    lbl_enc = LabelEncoder()
    full_data = pd.concat([train_df[features], test_df[features]],axis=0)
    for col in (features):
        print(col)
        lbl_enc.fit(full_data[col].values)
        train_df[col] = lbl_enc.transform(train_df[col])
        test_df[col] = lbl_enc.transform(test_df[col])
    return train_df, test_df

# Read the training and testing data


In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Add The Time Features

In [ ]:
train_df = train_df.sort_values(['date','device']).reset_index(drop=True) 
test_df = test_df.sort_values(['date','device']).reset_index(drop=True)

for dataset in (train_df,test_df):
    dataset['date'] = pd.to_datetime(dataset['date'])
    dataset['Day'] = dataset.date.dt.day
    dataset['Month'] = dataset.date.dt.month
    dataset['Year'] = dataset.date.dt.year
    dataset['DayOfWeek'] = dataset.date.dt.dayofweek
    dataset['DayOfYear'] = dataset.date.dt.dayofyear
    dataset['Week'] = dataset.date.dt.weekofyear
    dataset.set_index('date', inplace=True)

In [ ]:
ID = test_df['ID']
test_df.drop('ID',inplace=True,axis=1)
train_df.drop('ID',inplace=True,axis=1)

# Exploratory data analysis

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.describe()

In [ ]:
test_df.describe()

In [ ]:
#The cardinality of each catgorical feature (Training)
cat_cols = train_df.columns
for col in cat_cols:
    print(col, train_df[col].nunique())

In [ ]:
#The cardinality of each catgorical feature (Testing)
cat_cols = test_df.columns
for col in cat_cols:
    print(col, test_df[col].nunique())

# Data Preprocessing

# Missing Data 

In [ ]:
#missing data percentage (Training)
total = train_df.isnull().sum().sort_values(ascending=False)
percent_1 = train_df.isnull().sum()/train_df.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(60)

In [ ]:
#missing data percentage (Testing)
total = test_df.isnull().sum().sort_values(ascending=False)
percent_1 = test_df.isnull().sum()/test_df.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data

In [ ]:
train_df = train_df.bfill().ffill()
test_df = test_df.bfill().ffill()

# Feature Engineering

## - Lags Features

In [ ]:
def LAG(data,LagFeature,shift=1,NewFeatures=[]) :
    data[NewFeatures[0]]   = data[LagFeature]  - data[LagFeature].shift(shift)
    data[NewFeatures[1]]   = data[LagFeature].shift(shift)

num_feats = train_df.columns
num_feats = num_feats.drop(['Week','DayOfYear','DayOfWeek','Year','Month','Day','pm2_5','temp_mean','humidity','site_longitude','site_latitude','device'])

for feature in num_feats:
    LAG(train_df,LagFeature=f'{feature}',shift=1,NewFeatures=[f'{feature}_diff_Lag1',f'{feature}_Lag1'])
    LAG(test_df,LagFeature=f'{feature}',shift=1,NewFeatures=[f'{feature}_diff_Lag1',f'{feature}_Lag1'])

## - Combination Between Time Features

In [ ]:
for dataset in (train_df,test_df):
    dataset['Year_Month'] = dataset['Year'].astype(str) + '-' + dataset['Month'].astype(str)
    dataset['Year_Week'] = dataset['Year'].astype(str) + '-' + dataset['Week'].astype(str)
    dataset['Year_Month_Day'] = dataset['Year'].astype(str) + '-' + dataset['Month'].astype(str) + '-' + dataset['Day'].astype(str)
    
feats = ['Year_Month','Year_Week','Year_Month_Day']
train_df,test_df = label_enc(train_df,test_df,feats)

## - Aggregations Features

In [ ]:
DevicePM2_5Mean = dict(train_df.groupby('device')['pm2_5'].mean())
DevicePM2_5Std = dict(train_df.groupby('device')['pm2_5'].std())
DevicePM2_5Min = dict(train_df.groupby('device')['pm2_5'].min())
DevicePM2_5Max = dict(train_df.groupby('device')['pm2_5'].max())

for dataset in (train_df,test_df):
    dataset['DevicePM2_5Mean'] = dataset['device'].map(DevicePM2_5Mean)
    dataset['DevicePM2_5Std'] = dataset['device'].map(DevicePM2_5Std)
    dataset['DevicePM2_5Min'] = dataset['device'].map(DevicePM2_5Min)
    dataset['DevicePM2_5Max'] = dataset['device'].map(DevicePM2_5Max)

In [ ]:
def Agg(Feature):
    for dataset in (train_df,test_df):
        dataset[f'{Feature}PerMonth'] = dataset['Month'].map(dict(dataset.groupby('Month')[Feature].mean()))
        dataset[f'{Feature}PerWeek'] = dataset['Year_Week'].map(dict(dataset.groupby('Year_Week')[Feature].mean()))
        dataset[f'{Feature}PerDay'] = dataset['Year_Month_Day'].map(dict(dataset.groupby('Year_Month_Day')[Feature].mean()))
        
        dataset[f'{Feature}Month_std'] = dataset['Month'].map(dict(dataset.groupby('Month')[Feature].std()))
        dataset[f'{Feature}Week_std'] = dataset['Year_Week'].map(dict(dataset.groupby('Year_Week')[Feature].std()))
        dataset[f'{Feature}Day_std'] = dataset['Year_Month_Day'].map(dict(dataset.groupby('Year_Month_Day')[Feature].std()))
        
        dataset[f'{Feature}Month_min'] = dataset['Month'].map(dict(dataset.groupby('Month')[Feature].min()))
        dataset[f'{Feature}Week_min'] = dataset['Year_Week'].map(dict(dataset.groupby('Year_Week')[Feature].min()))
        dataset[f'{Feature}Day_min'] = dataset['Year_Month_Day'].map(dict(dataset.groupby('Year_Month_Day')[Feature].min()))
       
        dataset[f'{Feature}Month_max'] = dataset['Month'].map(dict(dataset.groupby('Month')[Feature].max()))
        dataset[f'{Feature}Week_max'] = dataset['Year_Week'].map(dict(dataset.groupby('Year_Week')[Feature].max()))
        dataset[f'{Feature}Day_max'] = dataset['Year_Month_Day'].map(dict(dataset.groupby('Year_Month_Day')[Feature].max()))
        
Agg('temp_mean')
Agg('humidity')

In [ ]:
train_df.drop(['Year_Month','Year_Week','Year_Month_Day'],inplace=True,axis=1)
test_df.drop(['Year_Month','Year_Week','Year_Month_Day'],inplace=True,axis=1)

## - Rolling Features

In [ ]:
def Rolling(feature):
    for dataset in (train_df,test_df):
        dataset[f'{feature}_Rolling_3'] = dataset[feature].rolling(3).mean()
        dataset[f'{feature}_Rolling_5'] = dataset[feature].rolling(5).mean()

        dataset[f"{feature}_rolling_mean_60"] = dataset.rolling(60).mean()[feature]
        dataset[f"{feature}_rolling_max_60"] = dataset.rolling(60).max()[feature]
        dataset[f"{feature}_rolling_min_60"] = dataset.rolling(60).min()[feature]

        dataset[f"{feature}_rolling_mean_30"] = dataset.rolling(30).mean()[feature]
        dataset[f"{feature}_rolling_max_30"] = dataset.rolling(30).max()[feature]
        dataset[f"{feature}_rolling_min_30"] = dataset.rolling(30).min()[feature]

        dataset[f"{feature}_rolling_mean_10"] = dataset.rolling(10).mean()[feature]
        dataset[f"{feature}_rolling_max_10"] = dataset.rolling(10).max()[feature]
        dataset[f"{feature}_rolling_min_10"] = dataset.rolling(10).min()[feature]

Rolling('temp_mean')
Rolling('humidity')

## - Polar Coordinates

In [ ]:
def Polar(X,y, a = 0, b = 0): # a and b represnt the center
    r = np.sqrt((X-a)**2 + (y-b)**2)
    phi = np.arctan2((y-a), (X-b))
    return r, phi

train_df['R'], train_df['Phi'] = Polar(train_df["site_latitude"],train_df["site_longitude"])
test_df['R'], test_df['Phi'] = Polar(test_df["site_latitude"],test_df["site_longitude"])

## - Foureier Frequnecies and Amplitudes For Features That Contain Seasonality

In [ ]:
freq2_dict_no_log = dict()
freq3_dict_no_log = dict()

amp2_dict_no_log = dict()
amp3_dict_no_log = dict()

for feat_1 in ('Year','Month','Day'):
    for feat_2 in ('temp_mean', 'humidity'):
        for i in range(min(train_df[feat_1].unique()), max(train_df[feat_1].unique()) + 1):

            a = train_df.loc[train_df[feat_1]==i]
            a_sales = a[feat_2]

            Y = np.fft.fft(a_sales.values)
            Y = abs(Y)
            freq = np.fft.fftfreq(len(Y), 1)

            intercept_index = np.argmax(Y)
            Y = np.delete(Y, intercept_index)
            freq = np.delete(freq, intercept_index)

            amplitude_1_index = np.argmax(Y)
            amplitude_1 = Y[amplitude_1_index]
            Y = np.delete(Y, amplitude_1_index)
            freq_1 = freq[amplitude_1_index]
            freq = np.delete(freq, amplitude_1_index)

            amplitude_2_index = np.argmax(Y)
            amplitude_2 = Y[amplitude_2_index]
            Y = np.delete(Y, amplitude_2_index)
            freq_2 = freq[amplitude_2_index]
            freq = np.delete(freq, amplitude_2_index)

            amplitude_3_index = np.argmax(Y)
            amplitude_3 = Y[amplitude_3_index]
            Y = np.delete(Y, amplitude_3_index)
            freq_3 = freq[amplitude_3_index]
            freq = np.delete(freq, amplitude_3_index)

            #Freq_1 is not included because it seems as it is always 0
            a[f'Frequency_2_{feat_1}_{feat_2}'] = freq_2
            a[f'Frequency_3_{feat_1}_{feat_2}'] = freq_3

            a[f'Amplitude_2_{feat_1}_{feat_2}'] = amplitude_2
            a[f'Amplitude_3_{feat_1}_{feat_2}'] = amplitude_3


            freq2_dict_no_log[i] = freq_2
            freq3_dict_no_log[i] = freq_3

            amp2_dict_no_log[i] = amplitude_2
            amp3_dict_no_log[i] = amplitude_3


            if i == min(train_df[feat_1].unique()):
                k = a
            else:
                k = pd.concat([k,a])
                
        train_df = k
        
        test_df[f'Frequency_2_{feat_1}_{feat_2}'] = test_df[feat_1].map(freq2_dict_no_log)
        test_df[f'Frequency_3_{feat_1}_{feat_2}'] = test_df[feat_1].map(freq3_dict_no_log)
        test_df[f'Amplitude_2_{feat_1}_{feat_2}'] = test_df[feat_1].map(amp2_dict_no_log)
        test_df[f'Amplitude_3_{feat_1}_{feat_2}'] = test_df[feat_1].map(amp3_dict_no_log)
        
        freq2_dict_no_log = dict()
        freq3_dict_no_log = dict()
        amp2_dict_no_log = dict()
        amp3_dict_no_log = dict()

## - Percentage change in Temperature and Humidity 

In [ ]:
periods = [1, 3, 7, 14]
for period in periods:
    train_df.loc[:, f"PctChangeTemp_{period}"] = train_df["temp_mean"].pct_change(period)
    train_df.loc[:, f"PctChangeHumi_{period}"] = train_df["humidity"].pct_change(period)
    test_df.loc[:, f"PctChangeTemp_{period}"] = test_df["temp_mean"].pct_change(period)
    test_df.loc[:, f"PctChangeHumi_{period}"] = test_df["humidity"].pct_change(period)

## - Historic Volatility

In [ ]:
periods = [3, 7, 14]
for period in periods:
    train_df.loc[:, f"volatility_temp_mean_{period}"] = train_df["temp_mean"].diff().rolling(period).std()
    test_df.loc[:, f"volatility_temp_mean_{period}"] = test_df["temp_mean"].diff().rolling(period).std()
    train_df.loc[:, f"volatility_humidity_{period}"] = train_df["humidity"].diff().rolling(period).std()
    test_df.loc[:, f"volatility_humidity_{period}"] = test_df["humidity"].diff().rolling(period).std()

# Encoding

In [ ]:
train_df, test_df = label_enc(train_df,test_df,['device'])

# Modeling

In [ ]:
#Averaging the predictions of the same model with different seeds to get more consistent results
X = train_df.drop('pm2_5',axis=1)
y = train_df['pm2_5']

Predictions = pd.DataFrame()

for seed in range(20,46):
    print(f'Seed: {seed}')
    params = {'n_estimators': 2064, 'learning_rate': 0.03196897706232692, 'depth': 5, 'reg_lambda': 12.680808984686983}
    CB = CatBoostRegressor(**params,verbose=0, random_state=seed, task_type = 'GPU')
    LogCB = TransformedTargetRegressor(CB, func = np.log1p, inverse_func = np.expm1)  
    LogCB.fit(X, y)

    Predictions[f'Target_{seed}'] = LogCB.predict(test_df)
    Predictions[f'Target_{seed}'] = Predictions[f'Target_{seed}'] * 0.975  #A Correction Factor of 0.975
    
#Averaging the Results
Predictions['Mean'] = Predictions.mean(axis=1)
Predictions['HMean'] = Predictions.apply(stats.hmean, axis=1)
Predictions['GMean'] = Predictions.apply(stats.gmean, axis=1)

#Averaging the Second Results
FinalPred = Predictions[['Mean','HMean','GMean']].apply(stats.hmean,axis=1)

#Making the submission file
submission = pd.DataFrame({"Id": ID ,"pm2_5": FinalPred.values})
submission.to_csv('AirQualityPrediction.csv',index=False)

## Show the Features Importances 

In [ ]:
CB.fit(X,y)
plotImp(CB,X)